In [1]:
# ============================================
# 설치 (Runpod A40 / 로컬)
# ============================================

# CPU 전용
# python -m pip install -U "torch>=2.2,<3.0" scikit-learn pandas numpy joblib sentence-transformers transformers

# GPU (CUDA 12.1, Runpod A40)
!python -m pip install -U "torch>=2.2,<3.0" torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!python -m pip install -U scikit-learn pandas numpy joblib sentence-transformers transformers

Looking in indexes: https://download.pytorch.org/whl/cu121
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 99.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.9/798.9 kB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.4/35.4 MB 46.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 76.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.2
    Uninstalling numpy-2.1.2:
      Successfully uninstalled numpy-2.1.2

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To upd

In [2]:
!pip uninstall transformers -y
!pip uninstall huggingface-hub -y
!pip uninstall tokenizers -y

!pip install --no-cache-dir transformers

Found existing installation: transformers 4.55.4
Uninstalling transformers-4.55.4:
  Successfully uninstalled transformers-4.55.4
Found existing installation: huggingface-hub 0.34.4
Uninstalling huggingface-hub-0.34.4:
  Successfully uninstalled huggingface-hub-0.34.4
Found existing installation: tokenizers 0.21.4
Uninstalling tokenizers-0.21.4:
  Successfully uninstalled tokenizers-0.21.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 239.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 377.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 482.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [4]:
# ============================================
# MiniLM 임베딩 + AutoModelForSequenceClassification + Threshold 최적화 + Epoch 8
# ============================================

import os, time, re
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score, classification_report

import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn import BCEWithLogitsLoss
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.optim import AdamW

# -------------------------------
# 설정
# -------------------------------
DATA_CSV = "perfumes_huggingface.csv"
MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
TOP_K = 3
RARE_MIN_COUNT = 7
MAX_LEN = 384
BATCH_SIZE = 16
EPOCHS = 8
LR = 1e-5

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"[Device] {device}")

# -------------------------------
# 유틸 함수
# -------------------------------
def split_labels(s: str):
    s = str(s)
    for sep in [",", "|", "/", ";"]:
        s = s.replace(sep, " ")
    return [t.strip() for t in s.split() if t.strip()]

# ✨ 전처리 함수 (불용어 단어 단위 매칭)
def preprocess_text(text: str, brand_list=None):
    text = str(text).lower()

    # 브랜드명 제거
    if brand_list:
        for brand in brand_list:
            text = re.sub(rf"\b{re.escape(brand.lower())}\b", " ", text)

    # 불필요한 특수문자 제거
    text = re.sub(r"[^a-zA-Z가-힣0-9\s]", " ", text)

    # 불용어 리스트
    stopwords = {
        # 관사
        "the", "a", "an",
    
        # 접속사
        "and", "or", "but", "if", "while", "though",
    
        # 전치사
        "of", "to", "in", "for", "on", "at", "by", "from", "into", "onto",
        "with", "about", "over", "under", "between", "through", "without",
    
        # 대명사
        "i", "me", "my", "mine", "you", "your", "yours",
        "he", "him", "his", "she", "her", "hers",
        "it", "its", "we", "us", "our", "ours",
        "they", "them", "their", "theirs",
    
        # 기타 불필요 단어
        "is", "am", "are", "was", "were", "be", "been", "being",
        "do", "does", "did", "done",
        "have", "has", "had", "having",
        "will", "would", "shall", "should", "can", "could", "may", "might", "must",
    
        # 한정사 / 지시어
        "this", "that", "these", "those", "such", "some", "any", "each", "every",
        "other", "another", "all", "both", "few", "many", "more", "most", "much", "no", "nor"
    }

    # 불용어 제거 (단어 단위 매칭)
    for sw in stopwords:
        text = re.sub(rf"\b{re.escape(sw)}\b", " ", text)

    # 다중 공백 정리
    text = re.sub(r"\s+", " ", text).strip()

    return text

# -------------------------------
# 1) 데이터 로드 & 전처리
# -------------------------------
df = pd.read_csv(DATA_CSV, sep="|", engine="python", on_bad_lines="skip")
df = df[~df["description"].isna()].copy()
df["labels"] = df["fragrances"].apply(split_labels)

# 브랜드명 리스트 자동 추출
brand_list = []
if "brand" in df.columns:
    brand_list = df["brand"].dropna().unique().tolist()
else:
    brand_list = []

# description 전처리 적용
df["description"] = df["description"].apply(lambda x: preprocess_text(x, brand_list))

# 희소 라벨 제거
cnt = Counter([l for L in df["labels"] for l in L])
rare = {k for k, v in cnt.items() if v <= RARE_MIN_COUNT}
df["labels"] = df["labels"].apply(lambda L: [l for l in L if l not in rare])
df = df[df["labels"].map(len) > 0].copy()

mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(df["labels"])

from sklearn.model_selection import train_test_split
X_train_text, X_val_text, y_train, y_val = train_test_split(
    df["description"].tolist(), Y, test_size=0.2, random_state=42
)

# -------------------------------
# 2) 토크나이저 및 데이터셋
# -------------------------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

class PerfumeDataset(Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenizer(texts, padding=True, truncation=True, max_length=MAX_LEN, return_tensors="pt")
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = PerfumeDataset(X_train_text, y_train)
val_dataset = PerfumeDataset(X_val_text, y_val)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

# -------------------------------
# 3) 모델 & 옵티마이저
# -------------------------------
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=Y.shape[1],
    problem_type="multi_label_classification"
).to(device)

optimizer = AdamW(model.parameters(), lr=LR)
loss_fn = BCEWithLogitsLoss()

# -------------------------------
# 4) 학습 루프
# -------------------------------
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for batch in train_loader:
        inputs = {k: v.to(device) for k, v in batch.items() if k != "labels"}
        labels = batch["labels"].to(device)

        outputs = model(**inputs)
        loss = loss_fn(outputs.logits, labels)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()

    print(f"Epoch {epoch+1} | Train Loss: {total_loss/len(train_loader):.4f}")

# -------------------------------
# 5) 검증 예측 & Threshold 최적화
# -------------------------------
model.eval()
all_logits, all_labels = [], []

with torch.no_grad():
    for batch in val_loader:
        inputs = {k: v.to(device) for k, v in batch.items() if k != "labels"}
        labels = batch["labels"].cpu().numpy()
        outputs = model(**inputs).logits.cpu().numpy()
        all_logits.append(outputs)
        all_labels.append(labels)

y_val_proba = torch.sigmoid(torch.tensor(np.vstack(all_logits))).numpy()
y_val = np.vstack(all_labels)

thresholds = {}
y_val_pred_opt = np.zeros_like(y_val)

for i, label in enumerate(mlb.classes_):
    best_thr, best_f1 = 0.5, -1
    for thr in np.linspace(0.2, 0.5, 16):
        pred = (y_val_proba[:, i] >= thr).astype(int)
        f1 = f1_score(y_val[:, i], pred, zero_division=0)
        if f1 > best_f1:
            best_thr, best_f1 = thr, f1
    thresholds[label] = best_thr
    y_val_pred_opt[:, i] = (y_val_proba[:, i] >= best_thr).astype(int)

print("\n[Best Thresholds per label]")
for k, v in thresholds.items():
    print(f"{k}: {v:.2f}")

# -------------------------------
# 6) 평가
# -------------------------------
print("\n=== Threshold-based ===")
print(f"Micro-F1: {f1_score(y_val, y_val_pred_opt, average='micro'):.4f}")
print(f"Macro-F1: {f1_score(y_val, y_val_pred_opt, average='macro'):.4f}")
print(f"Sample-F1: {f1_score(y_val, y_val_pred_opt, average='samples'):.4f}")
print("\n[classification_report @thr]")
print(classification_report(y_val, y_val_pred_opt, target_names=mlb.classes_, zero_division=0))

# Top-K 기반 예측도 비교
topk_preds = np.argsort(-y_val_proba, axis=1)[:, :TOP_K]
topk_bin = np.zeros_like(y_val)
for i, preds in enumerate(topk_preds):
    topk_bin[i, preds] = 1

print("\n=== Top-K-based ===")
print(f"Micro-F1: {f1_score(y_val, topk_bin, average='micro'):.4f}")
print(f"Macro-F1: {f1_score(y_val, topk_bin, average='macro'):.4f}")
print(f"Sample-F1: {f1_score(y_val, topk_bin, average='samples'):.4f}")

# -------------------------------
# 7) 예측 함수
# -------------------------------
def predict_multilingual(text: str, topk=3, thresholds=None):
    text = preprocess_text(text, brand_list)  # ✨ 입력도 전처리 적용
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=MAX_LEN).to(device)
    with torch.no_grad():
        logits = model(**inputs).logits
        proba = torch.sigmoid(logits).cpu().numpy()[0]

    if thresholds is not None:
        pick = [i for i, p in enumerate(proba) if p >= thresholds.get(mlb.classes_[i], 0.5)]
        if not pick:
            pick = np.argsort(-proba)[:topk]
    else:
        pick = np.argsort(-proba)[:topk]

    return [mlb.classes_[i] for i in pick]

# -------------------------------
# 예시 실행
# -------------------------------
print("\n[Example Prediction]")
print(predict_multilingual("바닷가에서 느껴지는 시원하고 약간 달콤한 향이 좋아요", topk=3, thresholds=thresholds))

[Device] cuda


tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1 | Train Loss: 0.5304
Epoch 2 | Train Loss: 0.3349
Epoch 3 | Train Loss: 0.2632
Epoch 4 | Train Loss: 0.2413
Epoch 5 | Train Loss: 0.2331
Epoch 6 | Train Loss: 0.2292
Epoch 7 | Train Loss: 0.2271
Epoch 8 | Train Loss: 0.2257

[Best Thresholds per label]
Amber: 0.20
Aromatic: 0.20
Blossom: 0.20
Bouquet: 0.20
Citrus: 0.20
Classical: 0.20
Crisp: 0.20
Dry: 0.20
Floral: 0.40
Flower: 0.20
Fougère: 0.20
Fresh: 0.20
Fresher: 0.20
Fruity: 0.20
Gourmand: 0.20
Green: 0.20
Iris: 0.20
Jasmine: 0.20
Lily: 0.20
Mossy: 0.20
Musk: 0.20
Orange: 0.20
Rich: 0.20
Richer: 0.20
Rose: 0.20
Soft: 0.20
Spicy: 0.20
Tuberose: 0.20
Valley: 0.20
Violet: 0.20
Water: 0.20
White: 0.20
Woods: 0.20
Woody: 0.20
of: 0.20
the: 0.20

=== Threshold-based ===
Micro-F1: 0.4230
Macro-F1: 0.0704
Sample-F1: 0.4225

[classification_report @thr]
              precision    recall  f1-score   support

       Amber       0.35      1.00      0.52       316
    Aromatic       0.00      0.00      0.00        83
     Blossom       

In [6]:
!pip install xgboost

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 MB 77.2 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
# ============================================
# MiniLM 임베딩 + XGBoost 분류기 + Threshold 최적화
# ============================================

import os, re
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

import torch
from transformers import AutoTokenizer, AutoModel

# -------------------------------
# 설정
# -------------------------------
DATA_CSV = "perfumes_huggingface.csv"
MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
TOP_K = 3
RARE_MIN_COUNT = 7
MAX_LEN = 256
BATCH_SIZE = 16

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"[Device] {device}")

# -------------------------------
# 유틸 함수
# -------------------------------
def split_labels(s: str):
    s = str(s)
    for sep in [",", "|", "/", ";"]:
        s = s.replace(sep, " ")
    return [t.strip() for t in s.split() if t.strip()]

# ✨ 전처리 함수 (불용어 단어 단위 매칭)
def preprocess_text(text: str, brand_list=None):
    text = str(text).lower()

    # 브랜드명 제거
    if brand_list:
        for brand in brand_list:
            text = re.sub(rf"\b{re.escape(brand.lower())}\b", " ", text)

    # 불필요한 특수문자 제거
    text = re.sub(r"[^a-zA-Z가-힣0-9\s]", " ", text)

    # 불용어 리스트
    stopwords = {
        # 관사
        "the", "a", "an",
    
        # 접속사
        "and", "or", "but", "if", "while", "though",
    
        # 전치사
        "of", "to", "in", "for", "on", "at", "by", "from", "into", "onto",
        "with", "about", "over", "under", "between", "through", "without",
    
        # 대명사
        "i", "me", "my", "mine", "you", "your", "yours",
        "he", "him", "his", "she", "her", "hers",
        "it", "its", "we", "us", "our", "ours",
        "they", "them", "their", "theirs",
    
        # 기타 불필요 단어
        "is", "am", "are", "was", "were", "be", "been", "being",
        "do", "does", "did", "done",
        "have", "has", "had", "having",
        "will", "would", "shall", "should", "can", "could", "may", "might", "must",
    
        # 한정사 / 지시어
        "this", "that", "these", "those", "such", "some", "any", "each", "every",
        "other", "another", "all", "both", "few", "many", "more", "most", "much", "no", "nor"
    }

    for sw in stopwords:
        text = re.sub(rf"\b{re.escape(sw)}\b", " ", text)

    text = re.sub(r"\s+", " ", text).strip()
    return text

# -------------------------------
# 1) 데이터 로드 & 전처리
# -------------------------------
df = pd.read_csv(DATA_CSV, sep="|", engine="python", on_bad_lines="skip")
df = df[~df["description"].isna()].copy()
df["labels"] = df["fragrances"].apply(split_labels)

# 브랜드명 리스트
brand_list = df["brand"].dropna().unique().tolist() if "brand" in df.columns else []

# 전처리
df["description"] = df["description"].apply(lambda x: preprocess_text(x, brand_list))

# 희소 라벨 제거
cnt = Counter([l for L in df["labels"] for l in L])
rare = {k for k, v in cnt.items() if v <= RARE_MIN_COUNT}
df["labels"] = df["labels"].apply(lambda L: [l for l in L if l not in rare])
df = df[df["labels"].map(len) > 0].copy()

mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(df["labels"])

X_train_text, X_val_text, y_train, y_val = train_test_split(
    df["description"].tolist(), Y, test_size=0.2, random_state=42
)

# -------------------------------
# 2) MiniLM 임베딩 추출
# -------------------------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
base_model = AutoModel.from_pretrained(MODEL_NAME).to(device)
base_model.eval()

def encode_texts(texts, batch_size=32):
    all_embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        enc = tokenizer(batch, padding=True, truncation=True, max_length=MAX_LEN, return_tensors="pt").to(device)
        with torch.no_grad():
            model_out = base_model(**enc)
            emb = model_out.last_hidden_state.mean(dim=1)  # [CLS] pooling 대신 평균 풀링
        all_embeddings.append(emb.cpu().numpy())
    return np.vstack(all_embeddings)

print("[Encoding Train Texts]")
X_train_emb = encode_texts(X_train_text, batch_size=BATCH_SIZE)
print("[Encoding Validation Texts]")
X_val_emb = encode_texts(X_val_text, batch_size=BATCH_SIZE)

# -------------------------------
# 3) XGBoost 분류기 학습 (OvR)
# -------------------------------
xgb = XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    use_label_encoder=False,
    tree_method="gpu_hist" if device=="cuda" else "hist"
)

# 멀티라벨은 각 라벨마다 One-vs-Rest 방식으로 학습
from sklearn.multiclass import OneVsRestClassifier
clf = OneVsRestClassifier(xgb, n_jobs=-1)
clf.fit(X_train_emb, y_train)

# -------------------------------
# 4) 검증 예측 & Threshold 최적화
# -------------------------------
y_val_proba = clf.predict_proba(X_val_emb)  # shape: (n_samples, n_classes)
y_val_proba = np.array(y_val_proba)

thresholds = {}
y_val_pred_opt = np.zeros_like(y_val)

for i, label in enumerate(mlb.classes_):
    best_thr, best_f1 = 0.5, -1
    for thr in np.linspace(0.2, 0.5, 16):
        pred = (y_val_proba[:, i] >= thr).astype(int)
        f1 = f1_score(y_val[:, i], pred, zero_division=0)
        if f1 > best_f1:
            best_thr, best_f1 = thr, f1
    thresholds[label] = best_thr
    y_val_pred_opt[:, i] = (y_val_proba[:, i] >= best_thr).astype(int)

print("\n[Best Thresholds per label]")
for k, v in thresholds.items():
    print(f"{k}: {v:.2f}")

# -------------------------------
# 5) 평가
# -------------------------------
print("\n=== Threshold-based ===")
print(f"Micro-F1: {f1_score(y_val, y_val_pred_opt, average='micro'):.4f}")
print(f"Macro-F1: {f1_score(y_val, y_val_pred_opt, average='macro'):.4f}")
print(f"Sample-F1: {f1_score(y_val, y_val_pred_opt, average='samples'):.4f}")
print("\n[classification_report @thr]")
print(classification_report(y_val, y_val_pred_opt, target_names=mlb.classes_, zero_division=0))

# Top-K 기반 예측도 비교
topk_preds = np.argsort(-y_val_proba, axis=1)[:, :TOP_K]
topk_bin = np.zeros_like(y_val)
for i, preds in enumerate(topk_preds):
    topk_bin[i, preds] = 1

print("\n=== Top-K-based ===")
print(f"Micro-F1: {f1_score(y_val, topk_bin, average='micro'):.4f}")
print(f"Macro-F1: {f1_score(y_val, topk_bin, average='macro'):.4f}")
print(f"Sample-F1: {f1_score(y_val, topk_bin, average='samples'):.4f}")

# -------------------------------
# 6) 예측 함수
# -------------------------------
def predict_multilingual(text: str, topk=3, thresholds=None):
    text = preprocess_text(text, brand_list)
    emb = encode_texts([text], batch_size=1)
    proba = clf.predict_proba(emb)[0]

    if thresholds is not None:
        pick = [i for i, p in enumerate(proba) if p >= thresholds.get(mlb.classes_[i], 0.5)]
        if not pick:
            pick = np.argsort(-proba)[:topk]
    else:
        pick = np.argsort(-proba)[:topk]

    return [mlb.classes_[i] for i in pick]

# -------------------------------
# 예시 실행
# -------------------------------
print("\n[Example Prediction]")
print(predict_multilingual("바닷가에서 느껴지는 시원하고 약간 달콤한 향이 좋아요", topk=3, thresholds=thresholds))


[Device] cuda
[Encoding Train Texts]
[Encoding Validation Texts]


/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [01:40:14] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [01:40:14] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [01:40:14] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: